# Day 1

## Imports and data loading

In [ ]:
from utils import get_input, load_data

day = 15


In [ ]:
get_input(day)


In [ ]:
data = load_data(day, list_type="line", number=False)
test_data = [
    "1163751742",
    "1381373672",
    "2136511328",
    "3694931569",
    "7463417111",
    "1319128137",
    "1359912421",
    "3125421639",
    "1293138521",
    "2311944581",
]
test_answer_1 = 40
test_answer_2 = 315


## Part one

In [ ]:
import networkx as nx


class Cave:
    def __init__(self, rows):
        """Turn the input into a directed graph where edge weight is the weight of
        the node at the end of that path."""
        self.rows = rows
        self.width = len(self.rows[0])
        self.height = len(self.rows)
        self.graph = nx.DiGraph()
        # Make the grid
        for j, y in enumerate(self.rows):
            for i, x in enumerate(y):
                self.graph.add_node((j, i), cost=x)

        # Add the edges to the grid
        for node in self.graph.nodes:
            if node[0] > 0:
                end = (node[0] - 1, node[1])
                self._add_edges(node, end)
            if node[0] < self.height - 1:
                end = (node[0] + 1, node[1])
                self._add_edges(node, end)
            if node[1] > 0:
                end = (node[0], node[1] - 1)
                self._add_edges(node, end)
            if node[1] < self.width - 1:
                end = (node[0], node[1] + 1)
                self._add_edges(node, end)

    def __repr__(self):
        """Print a slightly wonky grid of costs."""
        costs = []
        current = 0
        for node in self.graph.nodes:
            if node[0] != current:
                costs.append("\n")
                current += 1
            costs.append(str(self.graph.nodes[node]["cost"]))
        return " ".join(costs)

    def _add_edges(self, start, end):
        """Add bidirectional edge between 2 nodes, with cost set to cost of end edge."""
        self.graph.add_edge(start, end, cost=self.graph.nodes[end]["cost"])
        self.graph.add_edge(end, start, cost=self.graph.nodes[start]["cost"])

    def draw(self):
        """Draw the graph, including edge weights and node weights.

        Doesn't work if the graph is massive.
        """
        costs = nx.get_node_attributes(self.graph, "cost")
        edge_costs = nx.get_edge_attributes(self.graph, "cost")
        pos = nx.spring_layout(self.graph)
        nx.draw(self.graph, pos, with_labels=True, labels=costs)
        nx.draw_networkx_edge_labels(self.graph, pos, edge_labels=edge_costs)

    @classmethod
    def normal_grid(cls, input_rows):
        """Turn input data into a Cave object."""
        rows = [[int(i) for i in row] for row in input_rows]
        return cls(rows)

    @classmethod
    def big_grid(cls, input_rows):
        """Expand input data into a big Cave as required by part two."""
        rows = [[int(i) for i in row] for row in input_rows]
        # Extend the rows sideways
        width = len(rows[0])
        for i in range(4):
            for j, row in enumerate(rows):
                added = [r + 1 for r in row[-width:]]
                added = [a if a <= 9 else a - 9 for a in added]
                row.extend(added)
        # Extend the rows downwards
        new = []
        for i in range(5):
            for row in rows:
                added = [r + i for r in row]
                added = [a if a <= 9 else a - 9 for a in added]
                new.append(added)

        return cls(new)

    def get_shortest_path(self):
        """Run djikstra to find the shortest path."""
        return nx.shortest_path(
            self.graph,
            source=(0, 0),
            target=(self.height - 1, self.width - 1),
            weight="cost",
            method="dijkstra",
        )

    def measure_shortest_path(self):
        """Add up the node costs in the shortest path and subtract the start node."""
        path = self.get_shortest_path()
        start = self.graph.nodes[(0, 0)]["cost"]
        return sum([self.graph.nodes[node]["cost"] for node in path]) - start


In [ ]:
test_cave = Cave.normal_grid(test_data)
assert test_cave.measure_shortest_path() == test_answer_1

cave = Cave.normal_grid(data)
cave.measure_shortest_path()


## Part two

In [ ]:
big_test = Cave.big_grid(test_data)
assert big_test.measure_shortest_path() == test_answer_2

big = Cave.big_grid(data)
big.measure_shortest_path()
